# Structural analysis of discontinuous concepts

The purpose of this analysis notebook is to make an in-depth analysis of structures. This makes the basis for a large portion of the corresponding section in the final thesis paper.

## Coordinated concepts

Coordinations are the only thing marked in GENIA while CRAFT also contains other types. Therefore, GENIA will be used primarily in this part. The variable `corpus_name` can be set to either `genia` or `craft` to run with.

In [1]:
import utils  # a python module in the same dir as the notebooks

In [2]:
corpus_name = 'craft'

First, let's load the corpus, then extract concepts and their constituent structures.

In [3]:
import os
os.chdir(utils.ROOT)  # get to the root directory of the project

from datautils import dataio, annotations as anno

# load the corpora
corpus = dataio.load_corpus(corpus_name)

# not all docs in genia get Constituent annotations; if so, leave them out
if corpus_name.lower() == 'genia':
    corpus = [doc for doc in corpus if doc.get_annotations(anno.Constituent)]

Loading CRAFT corpus ...


100%|██████████| 78/78 [00:58<00:00,  1.33it/s]


In [4]:
concept_consts = [(concept, doc.get_annotations_at(concept.span, anno.Constituent)[0])
                  for doc in corpus
                  for concept in doc.get_annotations(anno.DiscontinuousConcept)
                  if len(concept.get_spanned_tokens()) < 10]

# for craft, keep only the ones with a conjunction in them
temp_list = []
for concept, const in concept_consts:
    present_pos = {t.mapped_pos() for t in const.get_tokens()}
    if 'c' in present_pos:
        temp_list.append((concept, const))
concept_consts = temp_list

Each constituent can spit out a treebank structure. Let's do that and put it all in dataframes.

In [5]:
import re

def skipped_structure(const, allowed_tokens):
    """Returns  a treebank structure of a Constituent with a # symbol in front of
    all tokens that are in allowed_tokens"""
    # note: craft constituents often have labels like NP-SBJ; keep only the first part
    return '(' + const.label.split('-')[0] + ' ' + ' '.join(
        skipped_structure(c, allowed_tokens) if isinstance(c, anno.Constituent)
        else c.mapped_pos() if c not in allowed_tokens
        else '#' + c.mapped_pos()
        for c in const.constituents
    ) + ')'

def unlabeled_structure(const, allowed_tokens):
    """Returns a simplified treebank structure of a Constituent with no constituent labels
    and with a # symbol in front of all tokens that are in allowed_tokens"""
    return_string = '(' + ' '.join(
        unlabeled_structure(c, allowed_tokens) if isinstance(c, anno.Constituent)
        else c.mapped_pos() if c not in allowed_tokens
        else '#' + c.mapped_pos()
        for c in const.constituents
        if not contains_empty_token(c)
    ) + ')'
    # remove all cases of single token constituents, e.g. (n)
    return re.sub('\(+(#?.)\)+', r'\1', return_string)

def contains_empty_token(const):
    if not isinstance(const, anno.Constituent):
        return False
    for c in const.constituents:
        if isinstance(c, anno.Token) and c.mapped_pos() == 'Ø':
            return True
    return False

In [6]:
import pandas as pd
from collections import defaultdict

full_structures = defaultdict(list)
skip_structures = defaultdict(list)
collapsed_full_structures = defaultdict(list)
collapsed_skip_structures = defaultdict(list)

for concept, const in concept_consts:
    struct = const.structure()  
    full_structures[struct].append(const)
    
    skip_struct = skipped_structure(const, concept.get_tokens())
    skip_structures[skip_struct].append(const)
    
    collapsed_full = unlabeled_structure(const, {})
    collapsed_full_structures[collapsed_full].append(const)
    
    collapsed_skip = unlabeled_structure(const, concept.get_tokens())
    collapsed_skip_structures[collapsed_skip].append(const)

In [7]:
n_dcs = len(concept_consts)

dfs = {}

for name, structure_dict in [('full', full_structures),
                             ('skip', skip_structures),
                             ('col_full', collapsed_full_structures),
                             ('col_skip', collapsed_skip_structures)]:
    
    data_dict = {'struct': [], 'pos-seq': [], 'count': [], '%': [], 'example': []}
    
    for struct, sample in structure_dict.items():
        data_dict['struct'].append(struct)
        data_dict['pos-seq'].append(''.join(c for c in struct if c.islower() or c in ','))
        data_dict['count'].append(len(sample))
        data_dict['%'].append(round(len(sample) / n_dcs * 100, 2))
        data_dict['example'].append(sample[0].get_covered_text())
        
    dfs[name] = pd.DataFrame(data_dict)

In [8]:
dfs['col_skip'].sort_values('count', ascending=False).head(50)

,struct,pos-seq,count,%,example
66,((#n c n) #n),ncnn,24,3.16,lung and diaphragm development
64,((#a c a) #n),acan,24,3.16,pulmonary and diaphragmatic development
34,(#n (n c #n)),nncn,19,2.50,cell growth and proliferation
159,((#n #- c n) - #n),ncnn,16,2.11,AlphaA- and AlphaB-crystallin
79,(t (#a c a) #n),tacan,14,1.84,the aspartic and glutamic acids
17,(t (#n c n) #n),tncnn,8,1.05,the CA2 and CA3 regions
186,((#a c n) #n),acnn,8,1.05,associative and reversal learning
72,(#a (n c #n)),ancn,7,0.92,cellular proliferation and apoptosis
10,(a (#n c n) #n),ancnn,7,0.92,new HDL and LDL particles
89,((c #a c a) #n),cacan,6,0.79,both embryonic and adult hematopoiesis


In [9]:
# for latex table
import re
csv_string = dfs['full'].sort_values('count', ascending=False).head(15).to_csv(sep='&')
csv_string = re.sub(r'\n\d+&', r'  \\\\ \\addlinespace\n', csv_string)  # line breaks and row numbers
csv_string = re.sub('^&', r'', csv_string)  # first &
csv_string = re.sub('%', r'\\%', csv_string)  # \% for latex
csv_string = re.sub('Ø', r'\\O', csv_string)  # \O for latex
csv_string = re.sub('#([A-Za-z\-]+)', r'\\textbf{\1}', csv_string)  # bold typeface used words 
csv_string = re.sub('&', r'\t&\t', csv_string)  # more space on delimiters

In [10]:
print(csv_string)

struct	&	pos-seq	&	count	&	\%	&	example  \\ \addlinespace
(NP (ADJP a c a) n)	&	acan	&	23	&	3.03	&	pulmonary and diaphragmatic development  \\ \addlinespace
(NP (NML n c n) n)	&	ncnn	&	18	&	2.37	&	lung and diaphragm development  \\ \addlinespace
(NP t (ADJP a c a) n)	&	tacan	&	13	&	1.71	&	the aspartic and glutamic acids  \\ \addlinespace
(NP (ADJP a c a) n n)	&	acann	&	12	&	1.58	&	white and brown fat adipocytes  \\ \addlinespace
(NP n (NML n c n))	&	nncn	&	11	&	1.45	&	cell growth and proliferation  \\ \addlinespace
(NP (UCP a c n) n)	&	acnn	&	8	&	1.05	&	associative and reversal learning  \\ \addlinespace
(NP (NML n - c n) - n)	&	ncnn	&	8	&	1.05	&	μ- and m-calpain  \\ \addlinespace
(NML (NML n - c n) - n)	&	ncnn	&	7	&	0.92	&	AlphaA- and AlphaB-crystallin  \\ \addlinespace
(NP t (NML n c n) n)	&	tncnn	&	6	&	0.79	&	the CA2 and CA3 regions  \\ \addlinespace
(NP a (ADJP a c a) n)	&	aacan	&	6	&	0.79	&	abnormal diaphragmatic and cardiac development  \\ \addlinespace
(NP a (NML n c n) n)	&	anc

In [11]:
collapsed_skip_structures['((#a c a) #n)']

[Constituent('pulmonary and diaphragmatic development'(12138, 12177))\NP,
 Constituent('embryonic and adult hematopoiesis'(5521, 5554))\NP,
 Constituent('Primitive and Definitive Erythropoiesis'(26255, 26294))\NP,
 Constituent('oral and corneal epithelia'(974, 1000))\NP,
 Constituent('eutherian and marsupial mammals'(5584, 5615))\NP,
 Constituent('slow and fast fibres'(43419, 43439))\NP,
 Constituent('associative and reversal learning'(752, 785))\NP,
 Constituent('associative and reversal learning'(2380, 2413))\NP,
 Constituent('associative and reversal learning'(5583, 5616))\NP,
 Constituent('cardiac and pharyngeal development'(5858, 5892))\NP,
 Constituent('4th and 6th PAAs'(14473, 14489))\NP,
 Constituent('4th and 6th PAAs'(18569, 18585))\NP,
 Constituent('cardiac and pharyngeal morphogenesis'(25825, 25861))\NP,
 Constituent('4th and 6th PAAs'(25979, 25995))\NP,
 Constituent('inner and outer segments'(17499, 17523))\NP,
 Constituent('bipolar and amacrine cells'(43892, 43918))\NP,
 C

We can capture more examples under fewer labels if we only use POS-tag sequences.

In [12]:
CROSS_COUNTING = ['concept', 'super', 'skip']  # order: concept, super, skip

MOD = '[an]'
SUPER_SEQ_COLLAPSERS = {
    # re.compile('n*(n,?)+cnn+'),
    # re.compile('a*(a,?)+caa*n+')
    #re.compile('[an]c[an]n'),  # base case: shared head
    #re.compile('[an]ncn'),  # base case: shared modifier
    #re.compile('([an],)+[an],?c[an]n'),  # exp case: enumeration of modifiers + shared head
    #re.compile('[an]+(n,)+n,?cn'),  # exp case: enumeration of heads with shared modifier
    #re.compile('[an]c[an][an]+n'),  # exp case: multi-word head
    #re.compile('[an]+[an]ncn'),  # exp case: shared pre-modifiers
    #re.compile(f'({MOD}n?,?)+c{MOD}*n'),  # all
    re.compile('.c..'),  # base case: shared head
    re.compile('..c.'),  # base case: shared modifier
    re.compile('[^,c]+.c.'),  # expanded cases: shared head
    re.compile('.c.[^,c]+'),  # expanded cases: shared modifier
    re.compile('[^,c]+.c.[^,c]+'),  # expanded cases: combined
    re.compile('([^,c]+,)+.+,?c.[^,c]*'),  # expanded case: enumerated heads
    re.compile('[^,c]*.c.(,[^,c]+)+'),  # expanded case: enumerated modifiers
    re.compile('[^c]+c[^c]+c[^c]+'),  # base case: shared head
}

CONCEPT_SEQ_COLLAPSERS = {
    re.compile('(a|n)+n')
}
SAMPLE_CATEGORIES = {'concept', 'super', 'skip', 'cross-count'}

def make_sequence(tokens, collapser=None):
    sequence = [t.mapped_pos() if not t.pos == '#' else '#'
                for t in tokens]
    sequence_str = ''.join(sequence)
    if not collapser:
        return sequence_str

    for pos_pattern in collapser:
        if pos_pattern.fullmatch(sequence_str):
            return pos_pattern.pattern
    return sequence_str  # sequence did not match a pattern

In [13]:
disc_concepts = [dc for doc in corpus
                 for dc in doc.get_annotations(anno.DiscontinuousConcept)
                 if 'c' in {t.mapped_pos() for t in dc.get_spanned_tokens()}
                 and len(dc.get_tokens()) < 10]

samples_after_category = {}
for ct in SAMPLE_CATEGORIES:
    samples_after_category[ct] = defaultdict(list)

for dc in disc_concepts:
    concept_tokens = dc.get_tokens()
    all_tokens = dc.get_spanned_tokens()
    skip_tokens = [t if t in concept_tokens  # actual token
                   else anno.Token(dc.document, (-1, -1), '#')  # skipped token
                   for t in all_tokens]

    cross_count_type = []
    cross_count_example = []

    concept_sequence = make_sequence(concept_tokens, CONCEPT_SEQ_COLLAPSERS)
    concept_text = dc.get_covered_text()
    samples_after_category['concept'][concept_sequence].append(dc)
    if 'concept' in CROSS_COUNTING:
        cross_count_type.append(concept_sequence)
        cross_count_example.append(dc)

    super_sequence = make_sequence(all_tokens, SUPER_SEQ_COLLAPSERS)
    super_text = dc.get_spanned_text()
    samples_after_category['super'][super_sequence].append(dc)
    if 'super' in CROSS_COUNTING:
        cross_count_type.append(super_sequence)
        cross_count_example.append(dc)

    skip_sequence = make_sequence(skip_tokens)
    samples_after_category['skip'][skip_sequence].append(dc)
    if 'skip' in CROSS_COUNTING:
        cross_count_type.append(skip_sequence)
        cross_count_example.append(dc)

    samples_after_category['cross-count'][tuple(cross_count_type)].append(
        tuple(cross_count_example)
    )

In [14]:
data_dict = {'type': [], 'count': [], '%': [], 'example': []}
for type_, items in samples_after_category['super'].items():
    data_dict['type'].append(type_)
    data_dict['count'].append(len(items))
    data_dict['%'].append(round(len(items) / len(disc_concepts) * 100, 2))
    data_dict['example'].append(items[0].get_spanned_text())
    
df = pd.DataFrame(data_dict)

In [15]:
df.sort_values('count', ascending=False).head(50)

,type,count,%,example
0,"[^,c]+.c.[^,c]+",202,31.61,differentiation of enucleating skin and blood ...
3,.c..,165,25.82,HDL and LDL particles
7,".c.[^,c]+",84,13.15,transcription and translation termination signals
5,..c.,80,12.52,blastocyst injection and implantation
1,"([^,c]+,)+.+,?c.[^,c]*",54,8.45,"differentiation of skin, hair-shaft, and blood..."
4,"[^,c]+.c.",31,4.85,formation of endoderm and somites
2,[^c]+c[^c]+c[^c]+,16,2.50,transcription-coupled repair of either UV-indu...
6,"[^,c]*.c.(,[^,c]+)+",2,0.31,"capillaries (panels F and H, arrows) of the kn..."
8,"n-n,n-a--n-,n-n-a--n-,cn-n-n,n-a--n-nptnn",1,0.16,"G1 (PIlow, BrdU-negative) (R1), S (BrdU-positi..."
9,"n-n-a--n-,cn-n-n,n-a--n-nptnn",1,0.16,"S (BrdU-positive) (R2), and G2/M (PIhi, BrdU-n..."


In [16]:
for item in samples_after_category['super']['[an]+[an]ncn']:
    print(*(t.get_covered_text() for t in item.get_spanned_tokens()))

In [ ]:
# for exp case: multi-word head: for multiple tokens before/after the conjunction,
# do the tokens go with the head or the second modifier?
df = dfs['col_full']
subset = df[df['pos-seq'].str.contains('^[an]+[an]ncn$')]
subset.head(25)

What about CC's?'

In [ ]:
all_concepts = [c for doc in corpus for c in doc.get_annotations(anno.Concept)
               if not isinstance(c, anno.DiscontinuousConcept) and not len(c) < 2]


In [ ]:
from collections import Counter
dist_of_types = Counter(c.pos_sequence() for c in all_concepts)

In [ ]:
dist_of_types.most_common(10)

In [ ]:
simple_count = sum(v for t, v in dist_of_types.items() if re.match('[an]n', t))

In [ ]:
simple_count / sum(dist_of_types.values())